In [1]:
import random
import os
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import tempfile
import math
from tqdm.notebook import tqdm
import copy
import shutil

In [2]:
from mmflow.apis import init_model, inference_model
from mmflow.datasets import visualize_flow, write_flow
import mmcv
config_file = r"C:\Users\hikar\Documents\pwcnet_ft_4x1_300k_sintel_final_384x768.py"
checkpoint_file = r"C:\Users\hikar\Documents\pwcnet_ft_4x1_300k_sintel_final_384x768.pth"

C:\Users\hikar\anaconda3\envs\mmediting_1\lib\site-packages\mmcv\__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
# fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
unverified_context = ssl._create_unverified_context()
UCF_DATA_FOLDER = r"C:\Users\hikar\Documents\UCF_101"

def list_ucf_videos():
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi)", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)
  
def path_modify(video):
  action_dir = video.split('_')[1]
  if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
      text_list = video.split('_')
      base_video_path = text_list[0]+ '_' + action_dir + '_' + text_list[2] + '_' + text_list[3]
      print(base_video_path)
      return base_video_path
  
  return video

In [5]:
def fetch_ucf_action_list():
  cache_list = []
  video_list = list_ucf_videos()
  for path in video_list:
    action_dir = path.split('_')[1]
    if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
    cache_list.append(action_dir)
  cache_list = set(cache_list)
  cache_list = list(cache_list)
  cache_list.sort()
  return cache_list

In [6]:
def flo_out(model,img1,img2,img3):#コードかえる
  

    
  base_path_1 = os.path.splitext(os.path.basename(img1))[0]
  base_path_2 = os.path.splitext(os.path.basename(img2))[0]
  base_path_3 = os.path.splitext(os.path.basename(img3))[0]

# D:\GR_data\capture_all_frame\Fixed\Archery\frames_v_Archery_g02_c01
#"D:\GR_data\write_flow_folder"

  flo_root_folder = r"F:\Users\hikar\GR_data\write_flow_folder" #変更可
  split_path_list = img1.split(os.sep)
  flo_dir_name = os.path.join(flo_root_folder,split_path_list[3],split_path_list[4],split_path_list[5])#変更可
  
  if not os.path.isdir(flo_dir_name):
    os.makedirs(flo_dir_name)
#     print(flo_dir_name)

  flo_name_12 = os.path.join(flo_dir_name,base_path_1+base_path_2+".flo")
#   print(flo_name_12)
  flo_name_23 = os.path.join(flo_dir_name,base_path_2+base_path_3+".flo")
#   print(flo_name_23)

  if not os.path.exists(flo_name_12):
    result_12 = inference_model(model, img1, img2)
    write_flow(result_12,flow_file = flo_name_12)
    
  if not os.path.exists(flo_name_23):
    result_23 = inference_model(model, img2, img3)
    write_flow(result_23,flow_file = flo_name_23)

  # visualize_flow(result_12, save_file='flow_map_12.png')
  # visualize_flow(-result_12,save_file = 'flow_map_reverse.png')
  # visualize_flow(result_23,save_file = "flow_map_23.png")
  return flo_name_12,flo_name_23

In [7]:
def remove_values_closeto_0(flow_map12,flow_map23):
#   print(flow_map12)
#   print(flow_map23)
    
  condition_12_1 = (flow_map12 < 0.01)
  condition_12_2 = (flow_map12 > -0.01)
  condition_12 = condition_12_1 & condition_12_2

  condition_23_1 = (flow_map23 < 0.01)
  condition_23_2 = (flow_map23 > 0.01)
  condition_23 = condition_23_1 & condition_23_2

  flow_map12[condition_12] = 0
  flow_map23[condition_23] = 0

  return flow_map12,flow_map23


def read_2_flo(flow12_path,flow23_path):
#   print(flow23_path)
  flow_map12 = cv2.readOpticalFlow(flow12_path)
  flow_map23 = cv2.readOpticalFlow(flow23_path)

  flow_map12,flow_map23 = remove_values_closeto_0(flow_map12,flow_map23)


  return flow_map12,flow_map23

In [8]:
def convert_nan_to_0(flow_magnitude):
  nans = np.isnan(flow_magnitude)
  if np.any(nans):
      nans = np.where(nans)
      flow_magnitude[nans] = 0.
  
  return flow_magnitude


def flow_magnitude_out(flow_map12,flow_map23):
  flow_map_12_channel_0 = flow_map12[:,:,0]
  flow_map_12_channel_1 = flow_map12[:,:,1]
  
  flow_map_23_channel_0 = flow_map23[:,:,0]
  flow_map_23_channel_1 = flow_map23[:,:,1]
  
  flow_magnitude_12, _ = cv2.cartToPolar(flow_map_12_channel_0, flow_map_12_channel_1)
  flow_magnitude_23, _ = cv2.cartToPolar(flow_map_23_channel_0, flow_map_23_channel_1)
  # A couple times, we've gotten NaNs out of the above...
  
  flow_magnitude_12 = convert_nan_to_0(flow_magnitude_12)
  flow_magnitude_23 = convert_nan_to_0(flow_magnitude_23)

  return flow_magnitude_12,flow_magnitude_23

In [9]:
def warp_second_frame_to_first(flow_map_origin,second_frame):
  flow_map = copy.copy(flow_map_origin)
  h,w = flow_map.shape[:2]
  flow_map[:,:,0] += np.arange(w)#変換後の座標を指定
  flow_map[:,:,1] += np.arange(h)[:,np.newaxis]
  img2 = cv2.imread(second_frame)
  warped_prevImg = cv2.remap(img2, flow_map, None, cv2.INTER_LINEAR)#INTER_NEAREST#INTER_AREA
#   cv2.imwrite("warped_first_frame.png",warped_prevImg)

  return warped_prevImg

# Vimeo three condition

In [10]:
def with_movement_greater_than_3(flow_map,flow_magnitude):#2#(o)
  first_bool = False
  more_number = np.count_nonzero(flow_magnitude > 3)#ハイパーパラメータ
  pixel_number = flow_map.shape[0] * flow_map.shape[1]# 240*320=76800
#   print(more_number)
#   print(pixel_number)
#   print(more_number/pixel_number * 100)
  if (more_number/pixel_number * 100) > 5:#ハイパーパラメータ
    first_bool = True

#  print("first condition is :{}".format(more_number/pixel_number * 100))
  return first_bool

In [11]:
def intensity_levels(warped_prevImg, prevImg_path):#4#(x)
  second_bool = False
  prevImg = cv2.imread(prevImg_path)
    
  img_diff = cv2.absdiff(warped_prevImg, prevImg)
#   print(img_diff.shape)
  sum_img_diff = np.sum(img_diff,axis = 2)
#   print(sum_img_diff.shape)
#   cv2.imwrite('sabun-result.jpg',img_diff)
  average_l1_distance = np.mean(sum_img_diff)
#   print(average_l1_distance)
  if average_l1_distance < 15:#ハイパーパラメータ
    second_bool = True
#   print("second condition is :{}".format(average_l1_distance))
  return second_bool

In [12]:
# def average_difference_v21_v23(flow_map12 ,flow_map23):#5#(o)
#   third_bool = False

#   flow_map_diff = flow_map23 - flow_map12
# #   print(flow_map23)
# #   print("---------------------")
# #   print(flow_map12)
#   # print("---------------------")
#   # print(flow_map_diff)
#   # print("---------------------")
# #   print(flow_map_diff.shape)
#   # print("---------------------")
    
#   flow_map_diff_c1 = flow_map_diff[:,:,0]
#   flow_map_diff_c2 = flow_map_diff[:,:,1]
#   flow_map_disagreement, _ = cv2.cartToPolar(flow_map_diff_c1, flow_map_diff_c2)
#   # print(flow_map_disagreement)
#   # print("---------------------")
#   # print(flow_map_disagreement)
#   flow_map_mean_disagreement = np.mean(flow_map_disagreement)
#   #print(flow_map_mean_disagreement)
#   # print(np.sqrt((flow_map_diff_c1[0][0])**2 + (flow_map_diff_c2[0][0])**2))

#   if flow_map_mean_disagreement < 1.2:#ハイパーパラメータ
#     third_bool = True
# #   print("third condition is :{}".format(flow_map_mean_disagreement))
#   return third_bool

In [13]:
def Fixed_list_ucf(classify = "Fixed",div = 0):
    cache_list = []
    action_list = fetch_ucf_action_list()

    classify_dir = os.path.join(r"D:\GR_data\capture_crop_256",classify)#変更可
    for action in action_list:
        action_path = os.path.join(classify_dir,action)
        each_video_list = os.listdir(action_path)
        cache_list.append(each_video_list)
    return np.hstack(cache_list)

In [14]:
def frame_folder_full_path(folder_name,classify):#ex:file_name = v_ApplyEyeMakeup_g01_c01
  #ex:folder_name = frames_v_BenchPress_g24_c04
  action_dir = folder_name.split('_')[2]
  folder_name = os.path.join(r"D:\GR_data\capture_crop_256",classify,action_dir,folder_name)#変更可
  # folder_name = "/content/drive/MyDrive/UCF101_every_frame/" + classify + "/"+ action_dir + folder_name
#   print(folder_name)
  return folder_name

In [15]:
def choice_three_frame(frame_folder_path,model):#()
#ex:frame_folder = D:\GR_data\capture_all_frame\Fixed\Archery\frames_v_Archery_g02_c01
  frame_count = 0
  triplet_dir_count = 0
  triplet_folder = r"D:\GR_data\frame_triplet_dir"#

  split_path_list = frame_folder_path.split(os.sep)
  flo_dir_name = os.path.join(triplet_folder,split_path_list[3],split_path_list[4],split_path_list[5])#変更可
#ex:flo_dir_name = D:\GR_data\frame_triplet_dir\frame_triplet_dir\Fixed\ApplyEyeMakeup\frames_v_ApplyEyeMakeup_g01_c01


  frame_list = os.listdir(frame_folder_path)#frame_folder内のframeのリスト
  sorted(frame_list)
#   print(frame_list)
  last_frame_num = frame_list[-1]
  last_frame_num = os.path.splitext(last_frame_num)[0]
  last_frame_num = int(last_frame_num)
#   print(last_frame_num)

#Set the first 3 frames
  img1 = os.path.join(frame_folder_path, str(frame_count).zfill(6)+ ".png")
#   print(img1)
  img2 = os.path.join(frame_folder_path, str(frame_count+1).zfill(6)+ ".png")
#   print(img2)
  img3 = os.path.join(frame_folder_path, str(frame_count+2).zfill(6)+ ".png")
#   print(img3)

  while True:
    base_path_1 = os.path.basename(img1)
    base_path_2 = os.path.basename(img2)
    base_path_3 = os.path.basename(img3)
    
    each_triplet_dir = os.path.join(flo_dir_name,str(triplet_dir_count).zfill(6))#tripletを格納するfolder
    
    flo1 ,flo2 = flo_out(model,img1,img2,img3)#imgからflowをoutしてそのファイルpathを返す
    flow_map12 ,flow_map23 = read_2_flo(flo1,flo2)
    flow_magnitude_12,flow_magnitude_23 = flow_magnitude_out(flow_map12,flow_map23)

    
    first_bool_12 = with_movement_greater_than_3(flow_map12,flow_magnitude_12)
    first_bool_23 = with_movement_greater_than_3(flow_map23,flow_magnitude_23)

    warped_prevImg2_to_1 = warp_second_frame_to_first(flow_map12,img2)
    warped_prevImg3_to_2 = warp_second_frame_to_first(flow_map23,img3)
    second_bool_12 = intensity_levels(warped_prevImg2_to_1,img1)
    second_bool_23 = intensity_levels(warped_prevImg3_to_2,img2)

#     third_bool = average_difference_v21_v23(flow_map12 ,flow_map23)
    #2と3の前方flowと1と2の後方flowのマイナスを比較する形に変えたほうがいい??
    
    if (first_bool_12 and first_bool_23) and (second_bool_12 and second_bool_23):
      #Trueなら3frameずらしてtripletのカウントを+1
      if not os.path.isdir(each_triplet_dir):
        os.makedirs(each_triplet_dir)
#         print(each_triplet_dir)
      if not os.path.exists(os.path.join(each_triplet_dir,base_path_1)):
        shutil.copy(img1,each_triplet_dir)
      if not os.path.exists(os.path.join(each_triplet_dir,base_path_2)):
        shutil.copy(img2,each_triplet_dir)
      if not os.path.exists(os.path.join(each_triplet_dir,base_path_3)):
        shutil.copy(img3,each_triplet_dir)
      
      frame_count += 3
      triplet_dir_count += 1
    
      if frame_count + 2 > last_frame_num:
        break
      img1 = os.path.join(frame_folder_path, str(frame_count).zfill(6)+ ".png")
      img2 = os.path.join(frame_folder_path, str(frame_count+1).zfill(6)+ ".png")
      img3 = os.path.join(frame_folder_path, str(frame_count+2).zfill(6)+ ".png")
    
    else:
      frame_count += 1

      if frame_count + 2 > last_frame_num:
        break
      img1 = os.path.join(frame_folder_path, str(frame_count).zfill(6)+ ".png")
      img2 = os.path.join(frame_folder_path, str(frame_count+1).zfill(6)+ ".png")
      img3 = os.path.join(frame_folder_path, str(frame_count+2).zfill(6)+ ".png")
    
  return

In [16]:
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

load checkpoint from local path: C:\Users\hikar\Documents\pwcnet_ft_4x1_300k_sintel_final_384x768.pth


In [17]:
#試験
# choice_three_frame(r"D:\GR_data\capture_crop_256\Fixed\PlayingGuitar\frames_v_PlayingGuitar_g22_c02",model)#変更可

In [18]:
classify = "Fixed" #Unfixed#Fixed\n",
fetch_frame_folder_list = Fixed_list_ucf()#Fixedの中からframeをfetchしてこなければならない\n",
# print(fetch_frame_folder_list)\n",
#↑これでframe_v_～～のfolderのlistまでは取り出せる\n",
for fetch_path in tqdm(fetch_frame_folder_list):
  frame_folder = frame_folder_full_path(fetch_path,classify)#capture_all_frameのフルパスを取得\n",
  choice_three_frame(frame_folder,model)
#   print(frame_folder)

  0%|          | 0/6831 [00:00<?, ?it/s]

C:\Users\hikar\anaconda3\envs\mmediting_1\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [19]:
#D:\GR_data\frame_triplet_dir\Fixed\BaseballPitch\frames_v_BaseballPitch_g22_c04

triplet_all_count = 0
root_dir = r"D:\GR_data\frame_triplet_dir"
classify = "Fixed" #Unfixed#Fixed
fetch_frame_folder_list = Fixed_list_ucf()
#↑これでframe_v_～～のfolderのlistまでは取り出せる

for fetch_path in tqdm(fetch_frame_folder_list):
  action_dir = fetch_path.split("_")[2]
  full_path = os.path.join(root_dir,classify,action_dir,fetch_path)
  triplet_all_count += len(os.listdir(full_path))

print(triplet_all_count)

  0%|          | 0/6831 [00:00<?, ?it/s]

FileNotFoundError: [WinError 3] 指定されたパスが見つかりません。: 'D:\\GR_data\\frame_triplet_dir\\Fixed\\ApplyEyeMakeup\\frames_v_ApplyEyeMakeup_g01_c01'